# Bootstrapping Swap Curves

# 👉 <a id = "top">Table of Contents</a> 👈 

# [1. Functions](#p1)

# [2. Bootstrap the OIS discount factor](#p2)

# [3. Bootstrap the LIBOR discount factor](#p3)

# [4. Forward swap rates](#p4)

In [13]:
import math
import numpy as np

import scipy.stats as si
from scipy.stats import norm

from scipy.optimize import fsolve
from scipy.optimize import brentq

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import plotly.graph_objects as go
import pandas as pd
import datetime as dt

import warnings

# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

In [14]:
OIS_fixed_leg_frequency = 360
IRS_fixed_leg_frequency = 180
IRS_floating_leg_frequency = 180

# <a id = "p1">1.</a>  <font color = "green"> Functions </font>  [back to table of contents](#top)

In [15]:
# Get number of days from the months and years

def get_days_years(data: pd.DataFrame):
    if "Tenor" not in data.columns:
        raise KeyError("Column 'Tenor' is missing from the DataFrame.")
    
    data["Days"] = np.nan
    data["Years"] = np.nan
    for row in range(len(data)):
        text = data.iloc[row,data.columns.get_loc("Tenor")]
        number = int(text[0:-1])
        unit = text[-1]
        if unit == "m":
            Day = number * 30
        elif unit == "y":
            Day = number * 360
        else:
            print (f"Row {row} has an error, Tenor should be in months or years.")
        data.iloc[row,data.columns.get_loc("Days")] = Day
        data.iloc[row,data.columns.get_loc("Years")] = Day/360
    return data 

In [ ]:
def plot_discount_curve(df: pd.DataFrame,
                        discount_curve: str,
                        title: str):
    """This function plots the bootstrapped discount curve of the IRS data using Plotly"""
    
    fig = go.Figure()

    # Line plot
    fig.add_trace(go.Scatter(
        x=df.index, 
        y=df[discount_curve], 
        mode='lines+markers',  # Both line and markers
        name="Discount Curve",
        line=dict(width=2), 
        marker=dict(size=6)
    ))

    # Layout customization
    fig.update_layout(
        title=title,
        xaxis_title="Tenor (years)",
        yaxis_title="Discount Factor",
        template="plotly_white",
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True)
    )

    # Show figure
    fig.show()

In [ ]:
def compute_forward_libor(delta_tenor: float,
                          previous_discount_factor: float,
                          current_discount_factor: float):
    
    return (1 / delta_tenor * (previous_discount_factor / current_discount_factor - 1))

# <a id = "p2">2.</a>  <font color = "green"> Bootstrap the OIS discount factor </font>  [back to table of contents](#top)

In [ ]:
OIS_data = pd.read_excel("IR Data.xlsx",
                         sheet_name="OIS",
                         usecols="A:C")
OIS_data

,Tenor,Product,Rate
0,6m,OIS,0.00250
1,1y,OIS,0.00300
2,2y,OIS,0.00325
3,3y,OIS,0.00335
4,4y,OIS,0.00350
5,5y,OIS,0.00360
6,7y,OIS,0.00400
7,10y,OIS,0.00450
8,15y,OIS,0.00500
9,20y,OIS,0.00525


In [ ]:
get_days_years(OIS_data)

,Tenor,Product,Rate,Days,Years
0,6m,OIS,0.00250,180.0,0.5
1,1y,OIS,0.00300,360.0,1.0
2,2y,OIS,0.00325,720.0,2.0
3,3y,OIS,0.00335,1080.0,3.0
4,4y,OIS,0.00350,1440.0,4.0
5,5y,OIS,0.00360,1800.0,5.0
6,7y,OIS,0.00400,2520.0,7.0
7,10y,OIS,0.00450,3600.0,10.0
8,15y,OIS,0.00500,5400.0,15.0
9,20y,OIS,0.00525,7200.0,20.0


$$
D(0,T_n) = \frac{1 - r \cdot \sum_{i=1}^{n-1} D(0,T_i) \cdot \Delta_i}{1 + r \cdot \Delta_n}
$$

In [ ]:
OIS_data["OIS_Discount_Factor"] = np.nan
OIS_data["Sum_Discount_Factor"] = np.nan

'''
Core OIS discount factor bootstrapping loop
For each tenor:
If it's short-term (e.g. ≤ 1 year), treat it like a deposit or short OIS.
If it's longer, treat it like a par swap and use prior bootstrapped values to solve for the next discount factor.
'''

for row in range(len(OIS_data)):
    days = OIS_data.iloc[row,OIS_data.columns.get_loc("Days")]
    years = OIS_data.iloc[row,OIS_data.columns.get_loc("Years")]
    rate = OIS_data.iloc[row,OIS_data.columns.get_loc("Rate")]
    # If tenor is less than or equal to fixed leg frequency
    if days <= OIS_fixed_leg_frequency:         # Use simple interest formula
        OIS_data.iloc[row,OIS_data.columns.get_loc("OIS_Discount_Factor")] = \
        (
            1/(1+days/360
            *
            rate)
        )
        
        if years >= OIS_fixed_leg_frequency/360:
            OIS_data.iloc[row,OIS_data.columns.get_loc("Sum_Discount_Factor")] = \
                (
                    OIS_data.iloc[row,OIS_data.columns.get_loc("OIS_Discount_Factor")]
                )
    
    # If tenor is more than fixed leg frequency
    else:
        OIS_data.iloc[row,OIS_data.columns.get_loc("OIS_Discount_Factor")] = \
            (
                (
                    1  
                    -
                    rate
                    * 
                    (OIS_data.iloc[row-1,OIS_data.columns.get_loc("Sum_Discount_Factor")])
                    -
                    ((years-OIS_data.iloc[row-1,OIS_data.columns.get_loc("Years")]-1)*0.5)
                    *
                    rate
                    *
                    (OIS_data.iloc[row-1,OIS_data.columns.get_loc("OIS_Discount_Factor")])
                ) 
                / 
                (
                    1 + (1+(years-OIS_data.iloc[row-1,OIS_data.columns.get_loc("Years")]-1)*0.5)*rate
                )
            )
        
        if years - OIS_data.iloc[row-1,OIS_data.columns.get_loc("Years")] == 1:
            OIS_data.iloc[row,OIS_data.columns.get_loc("Sum_Discount_Factor")] = \
                (
                    (
                        years 
                        - 
                        OIS_data.iloc[row-1,OIS_data.columns.get_loc("Years")]
                    )
                    *
                    OIS_data.iloc[row,OIS_data.columns.get_loc("OIS_Discount_Factor")] 
                    + 
                    OIS_data.iloc[row-1,OIS_data.columns.get_loc("Sum_Discount_Factor")]
                )
        else:
            OIS_data.iloc[row,OIS_data.columns.get_loc("Sum_Discount_Factor")] = \
                (
                    (
                        years 
                        - 
                        OIS_data.iloc[row-1,OIS_data.columns.get_loc("Years")]+1
                    )
                    *
                    (
                        OIS_data.iloc[row,OIS_data.columns.get_loc("OIS_Discount_Factor")] 
                        +
                        OIS_data.iloc[row-1,OIS_data.columns.get_loc("OIS_Discount_Factor")]
                    )/2 
                    - OIS_data.iloc[row-1,OIS_data.columns.get_loc("OIS_Discount_Factor")]
                    + 
                    OIS_data.iloc[row-1,OIS_data.columns.get_loc("Sum_Discount_Factor")]
                )
OIS_data

,Tenor,Product,Rate,Days,Years,OIS_Discount_Factor,Sum_Discount_Factor
0,6m,OIS,0.00250,180.0,0.5,0.998752,NaN
1,1y,OIS,0.00300,360.0,1.0,0.997009,0.997009
2,2y,OIS,0.00325,720.0,2.0,0.993531,1.990540
3,3y,OIS,0.00335,1080.0,3.0,0.990015,2.980555
4,4y,OIS,0.00350,1440.0,4.0,0.986117,3.966672
5,5y,OIS,0.00360,1800.0,5.0,0.982184,4.948856
6,7y,OIS,0.00400,2520.0,7.0,0.972406,6.898556
7,10y,OIS,0.00450,3600.0,10.0,0.955977,9.782916
8,15y,OIS,0.00500,5400.0,15.0,0.927611,14.477704
9,20y,OIS,0.00525,7200.0,20.0,0.900076,19.033155


In [ ]:
OIS_Interpolated_Data = OIS_data[["Years", 'Rate', 'OIS_Discount_Factor']].copy()

# Generate the full sequence of 0.5 year steps
full_sequence = np.arange(0.5, 30.5, 0.5)

# Convert 'years' column of your DataFrame to a list
existing_years = OIS_Interpolated_Data['Years'].tolist()

# Remove the existing years from the full sequence
new_years = [year for year in full_sequence if year not in existing_years]

# Add new rows for interpolation
additional_years = pd.DataFrame({
    'Years': new_years,
    'Rate': [np.nan]*len(new_years),
    'OIS_Discount_Factor': [np.nan]*len(new_years)
})

# Concat the new_years dataframe with the existing dataframe with rates data
OIS_Interpolated_Data = pd.concat([additional_years, OIS_Interpolated_Data])

# Sort by ascending years
OIS_Interpolated_Data = OIS_Interpolated_Data.sort_values(by='Years')

# Linear interpolation for both the rates and OIS discount factors
OIS_Interpolated_Data = OIS_Interpolated_Data.interpolate(method='linear')

# Set index for plotting
OIS_Interpolated_Data.set_index("Years",inplace=True)

In [ ]:
OIS_Interpolated_Data

,Rate,OIS_Discount_Factor
Years,,
0.5,0.002500,0.998752
1.0,0.003000,0.997009
1.5,0.003125,0.995270
2.0,0.003250,0.993531
2.5,0.003300,0.991773
3.0,0.003350,0.990015
3.5,0.003425,0.988066
4.0,0.003500,0.986117
4.5,0.003550,0.984150


In [ ]:
plot_discount_curve(df=OIS_Interpolated_Data,
                    discount_curve="OIS_Discount_Factor",
                    title="OIS Discount Curve")

# <a id = "p3">3.</a>  <font color = "green"> Bootstrap the LIBOR discount factor </font>  [back to table of contents](#top)

In [ ]:
IRS_data = pd.read_excel("IR Data.xlsx",
                        sheet_name="IRS",
                        usecols="A:C")
IRS_data

,Tenor,Product,Rate
0,6m,LIBOR,0.0250
1,1y,IRS,0.0280
2,2y,IRS,0.0300
3,3y,IRS,0.0315
4,4y,IRS,0.0325
5,5y,IRS,0.0330
6,7y,IRS,0.0350
7,10y,IRS,0.0370
8,15y,IRS,0.0400
9,20y,IRS,0.0450


In [ ]:
get_days_years(IRS_data)

,Tenor,Product,Rate,Days,Years
0,6m,LIBOR,0.0250,180.0,0.5
1,1y,IRS,0.0280,360.0,1.0
2,2y,IRS,0.0300,720.0,2.0
3,3y,IRS,0.0315,1080.0,3.0
4,4y,IRS,0.0325,1440.0,4.0
5,5y,IRS,0.0330,1800.0,5.0
6,7y,IRS,0.0350,2520.0,7.0
7,10y,IRS,0.0370,3600.0,10.0
8,15y,IRS,0.0400,5400.0,15.0
9,20y,IRS,0.0450,7200.0,20.0


In [ ]:
IRS_Interpolated_Data = IRS_data[['Years', 'Rate']].copy()

# Generate the full sequence
full_sequence = np.arange(0.5, 30.5, 0.5)

# Convert 'years' column of your DataFrame to a list
existing_years = IRS_Interpolated_Data['Years'].tolist()

# Remove the existing years from the full sequence
new_years = [years for years in full_sequence if years not in existing_years]

# Create a dummy dataframe of the new_years along with nan for their rates
new_years = pd.DataFrame({
    'Years': new_years,
    'Rate': [np.nan]*len(new_years),
})

# Concat the new_years dataframe with the existing dataframe with rates data
IRS_Interpolated_Data = pd.concat([new_years, IRS_Interpolated_Data])

# Sort by ascending years
IRS_Interpolated_Data = IRS_Interpolated_Data.sort_values(by='Years')

# Set index for plotting
IRS_Interpolated_Data.set_index("Years",inplace=True)

In [ ]:
# Copy the OIS discount factors into the IRS data, as it is collateralized in cash and overnight interest is paid on collateral
IRS_Interpolated_Data['OIS_Discount_Factor'] = OIS_Interpolated_Data['OIS_Discount_Factor']
IRS_Interpolated_Data['IRS_Discount_Factor'] = np.nan
IRS_Interpolated_Data = IRS_Interpolated_Data.reset_index(names="Years")
IRS_Interpolated_Data

,Years,Rate,OIS_Discount_Factor,IRS_Discount_Factor
0,0.5,0.0250,0.998752,NaN
1,1.0,0.0280,0.997009,NaN
2,1.5,NaN,0.995270,NaN
3,2.0,0.0300,0.993531,NaN
4,2.5,NaN,0.991773,NaN
5,3.0,0.0315,0.990015,NaN
6,3.5,NaN,0.988066,NaN
7,4.0,0.0325,0.986117,NaN
8,4.5,NaN,0.984150,NaN
9,5.0,0.0330,0.982184,NaN


In [ ]:
# Get the index of the rows that have valid swap rates
index = IRS_Interpolated_Data[pd.notna(IRS_Interpolated_Data['Rate'])].index
index

Index([0, 1, 3, 5, 7, 9, 13, 19, 29, 39, 59], dtype='int64')

In [ ]:
# For each row with a valid IRS rate
for i in index:
    def equation1(df, i = i):
        # Copy the DataFrame up to that point
        irs = IRS_Interpolated_Data.iloc[0:i+1].copy()    
        
        # Set the IRS_Discount_Factor for the current row to df (which will be solved using brentq
        irs['IRS_Discount_Factor'].iloc[-1] = df
        
        # Interpolate the missing IRS discount factors linearly         
        irs[['Years', 'IRS_Discount_Factor']] = irs[['Years', 'IRS_Discount_Factor']].interpolate(method='linear')  
        
        # Compute the present value of the fixed leg payments for each swap by using the OIS discount factor and IRS discount factors.
        sum = 0
        for i in range(len(irs)):
            if i == 0:
                sum += irs["OIS_Discount_Factor"].iloc[i]/irs['IRS_Discount_Factor'].iloc[i]*2
            else:
                sum += irs["OIS_Discount_Factor"].iloc[i]*irs['IRS_Discount_Factor'].iloc[i-1]/irs['IRS_Discount_Factor'].iloc[i]*2
    
        # Minimize the difference between the present value of the fixed leg and the known swap rate. 
        return sum - (irs['Rate'].iloc[-1] + 2)*np.sum(irs['OIS_Discount_Factor'])

    IRS_Interpolated_Data.at[i, 'IRS_Discount_Factor'] = brentq(equation1, 0.001, 1)

IRS_Interpolated_Data = IRS_Interpolated_Data.interpolate(method='linear')
IRS_Interpolated_Data.set_index("Years",inplace=True)
IRS_Interpolated_Data

,Rate,OIS_Discount_Factor,IRS_Discount_Factor
Years,,,
0.5,0.025000,0.998752,0.987654
1.0,0.028000,0.997009,0.972577
1.5,0.029000,0.995270,0.957378
2.0,0.030000,0.993531,0.942179
2.5,0.030750,0.991773,0.926330
3.0,0.031500,0.990015,0.910482
3.5,0.032000,0.988066,0.894731
4.0,0.032500,0.986117,0.878981
4.5,0.032750,0.984150,0.863985


$$
\text{Forward LIBOR}_n = \left( \frac{\left( 1 + \frac{\text{Rate}_n}{2} \right)^{2T_n}}{\left( 1 + \frac{\text{Rate}_{n-1}}{2} \right)^{2T_{n-1}}} \right)^2 - 1
$$

Where:
- \(\text{Rate}_n\) is the **swap rate** for the current period.
- \(\text{Rate}_{n-1}\) is the **swap rate** for the previous period.
- \(T_n\) is the **tenor** for the current period in years.
- The rates are divided by 2 to account for semi-annual payments.

In [ ]:
# Calculation of the forward LIBOR rates 

# Initialization of Forward LIBOR Column
IRS_Interpolated_Data["Forward_LIBOR"]=np.nan

# Setting the first forward LIBOR rate equal to the swap rate for the first period. 
IRS_Interpolated_Data.iloc[0,IRS_Interpolated_Data.columns.get_loc("Forward_LIBOR")] = \
    (
        IRS_Interpolated_Data.iloc[0,IRS_Interpolated_Data.columns.get_loc("Rate")]
    )

# Iterating Over the Remaining Rows
for row in range(1,len(IRS_Interpolated_Data)):
    IRS_Interpolated_Data.iloc[row,IRS_Interpolated_Data.columns.get_loc("Forward_LIBOR")]=\
        (
            (
                (1+IRS_Interpolated_Data.iloc[row,IRS_Interpolated_Data.columns.get_loc("Rate")]/2)
                **
                (IRS_Interpolated_Data.index[row]*2)
                /
                (1+IRS_Interpolated_Data.iloc[row-1,IRS_Interpolated_Data.columns.get_loc("Rate")]/2)
                **
                (IRS_Interpolated_Data.index[row-1]*2)
            )**2
        )-1

In [ ]:
IRS_Interpolated_Data

,Rate,OIS_Discount_Factor,IRS_Discount_Factor,Forward_LIBOR
Years,,,,
0.5,0.025000,0.998752,0.987654,0.025000
1.0,0.028000,0.997009,0.972577,0.031245
1.5,0.029000,0.995270,0.957378,0.031242
2.0,0.030000,0.993531,0.942179,0.033275
2.5,0.030750,0.991773,0.926330,0.034038
3.0,0.031500,0.990015,0.910482,0.035565
3.5,0.032000,0.988066,0.894731,0.035309
4.0,0.032500,0.986117,0.878981,0.036328
4.5,0.032750,0.984150,0.863985,0.035053


In [ ]:
plot_discount_curve(df=IRS_Interpolated_Data,
                    discount_curve="IRS_Discount_Factor",
                    title="IRS Discount Curve")

# <a id = "p4">4.</a>  <font color = "green"> Forward swap rates </font>  [back to table of contents](#top)

In [ ]:
IRS_Interpolated_Data.reset_index(names="Years",inplace=True)
IRS_Interpolated_Data

,Years,Rate,OIS_Discount_Factor,IRS_Discount_Factor,Forward_LIBOR
0,0.5,0.025000,0.998752,0.987654,0.025000
1,1.0,0.028000,0.997009,0.972577,0.031245
2,1.5,0.029000,0.995270,0.957378,0.031242
3,2.0,0.030000,0.993531,0.942179,0.033275
4,2.5,0.030750,0.991773,0.926330,0.034038
5,3.0,0.031500,0.990015,0.910482,0.035565
6,3.5,0.032000,0.988066,0.894731,0.035309
7,4.0,0.032500,0.986117,0.878981,0.036328
8,4.5,0.032750,0.984150,0.863985,0.035053
9,5.0,0.033000,0.982184,0.848989,0.035562


In [ ]:
Forward_swap_rates_df = pd.DataFrame({'Start' : [1]*5 + [5]*5 + [10]*5,
                                    'Tenor' : [1,2,3,5,10]*3, 
                                    'Forward_Swap_Rates': [np.nan]*15})

for i in Forward_swap_rates_df.index:
    def equation(S, 
                 start = Forward_swap_rates_df.at[i, 'Start'], 
                 tenor = Forward_swap_rates_df.at[i, 'Tenor'], 
                 irs = IRS_Interpolated_Data):
        
        # Create a subset irs1 that only contains the rows from IRS_Interpolated_Data where the Years column falls within the start and start + tenor period.
        irs1 = irs[(irs['Years'] > start)&(irs['Years'] <= start+tenor)].copy()

        # Calculating the Fixed Leg Sum
        sum1 = 0
        for i in irs1.index:
                sum1 += \
                    (
                        irs.at[i, 'OIS_Discount_Factor']
                        *
                        irs.at[i-1, 'IRS_Discount_Factor']
                        /
                        irs.at[i, 'IRS_Discount_Factor']
                        *
                        2
                    )
        
        # Calculating the Floating Leg Sum
        sum2 = (S+2)*np.sum(irs1['OIS_Discount_Factor'])
        
        # Return the Difference for Root-Finding
        return sum1-sum2

    Forward_swap_rates_df.iloc[i, Forward_swap_rates_df.columns.get_loc("Forward_Swap_Rates")] = brentq(equation, 0.0001, 0.5)

In [ ]:
Forward_swap_rates_df

,Start,Tenor,Forward_Swap_Rates
0,1,1,0.032007
1,1,2,0.033259
2,1,3,0.034011
3,1,5,0.035255
4,1,10,0.038428
5,5,1,0.039274
6,5,2,0.040075
7,5,3,0.040072
8,5,5,0.041093
9,5,10,0.043634


In [ ]:
Forward_swap_rates_df.to_csv("Forward_swap_rates_df.csv",index=False)

In [ ]:
IRS_Interpolated_Data.to_csv("Discount_Factors.csv",index=False)